In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV




In [2]:
#загрузка данных из 2х категорий
newsgroups = fetch_20newsgroups(subset='all',categories=['alt.atheism', 'sci.space'])

In [3]:
#построение числового представления слов
#с помощью алгоритма TF-IDF
vectorizer = TfidfVectorizer()
#не забываем нормализовать обучающую выборку
#в данном случае обуч выб = train+test (так как subset='all')
X_scaled = vectorizer.fit_transform(newsgroups.data)
y = newsgroups.target

#кросс-валидация по 5 блокам
kf = KFold(n_splits = 5, shuffle = True, random_state=241)
#print(kf)
#будет обучение классификатора с линейным ядром с параметром С
#который пока неизвестен
clf = SVC(kernel='linear', random_state=241)
#ищем оптимальный минимальный С в отрезке [10^-5; 10^5]
grid = {'C': np.power(10.0, np.arange(-5, 6))}
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=kf, n_jobs=-1)
gs.fit(X_scaled, y)
#получение того самого С
gs.best_params_ 
#Можно посмотреть вручню на С и выбрать
#for a in gs.grid_scores_:
 #   print(a.mean_validation_score) # — оценка качества по кросс-валидации
  #  print(a.parameters) # — значения параметров


{'C': 1.0}

In [4]:
#обучение классификатора с найденным С=1
clf = SVC(C=1, kernel='linear', random_state=241)
clf.fit(X_scaled, y)


SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=241,
  shrinking=True, tol=0.001, verbose=False)

In [124]:
#abs()т.к. того требует задача (веса могут быть отрицательны, так работает алгоритм)
values = abs(clf.coef_.toarray()[0])
#(класс, к которому отнесено слово;числовое представление слова) вес слова
#print(clf.coef_)
#получение индексов наиболее важных 10 слов
top10 = np.argsort(values)[-10:] 
print(top10)
#22936 15606  5776 21850 23673 17802  5093  5088 12871 24019

[22936 15606  5776 21850 23673 17802  5093  5088 12871 24019]


In [127]:
#получение признаков - слова в тексте
feature_mapping = vectorizer.get_feature_names()
#получение 10 слов, которым соответствует j-й признах из массива с индексами признаков
l = [feature_mapping[j] for j in top10]
print(sorted(l))
#как понял - нашли наиболее значимые слова из текстов 2х категорий

['atheism', 'atheists', 'bible', 'god', 'keith', 'moon', 'religion', 'sci', 'sky', 'space']
